In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import random
from ortools.sat.python import cp_model
import time

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
#start_time = time.time()

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = '..\\eua-dataset\\users\\'
U = pd.read_csv(users_path + 'users-test.csv') # test with 8 users
#U = pd.read_csv(users_path + 'users-test-more-users.csv') # test with 14 users
#U = pd.read_csv(users_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = '..\\eua-dataset\\edge-servers\\'
S = pd.read_csv(servers_path + 'serverstest.csv') # test with 3 servers
#S = pd.read_csv(servers_path + 'serverstest-more-servers.csv') # test with 6 servers
#S = pd.read_csv(servers_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:
#-----------------------------------VARIABLES-----------------------------------

U_si = [] # Set of individuals allocated to server s_i

# Memory of function m_f
m1 = 1
m2 = 2
m3 = 5
m4=8
mem = [m1,m2,m3,m4]
W = []
for j in range(len(U)):
  m = random.choice([m1, m2, m3,m4])
  W.append(m)

index = sorted(range(len(W)), key=lambda a: W[a]) # Sort the users by their requirements


F=np.zeros([len(U),len(mem)])
for m in range(len(mem)):
  for j in range(len(U)):
    if W[j]==mem[m]:
      F[j][m]=1
    else:
      F[j][m]=0    

# Each server has memory capacity c
mu, sigma = 35, 10 # mean and standard deviation


M = [8,9,10]
""" for j in range(len(S)):
  norm_dist = np.random.normal(mu, sigma**2, 1)
  c = norm_dist.astype(int)
  for i in range(len(c)):
    if c[i] < 0:
      c[i] = 1 # Value suggested by MCF paper in case of negative values
  M.append(c)  """


' for j in range(len(S)):\n  norm_dist = np.random.normal(mu, sigma**2, 1)\n  c = norm_dist.astype(int)\n  for i in range(len(c)):\n    if c[i] < 0:\n      c[i] = 1 # Value suggested by MCF paper in case of negative values\n  M.append(c)  '

In [4]:
#-----------------------------------COVERAGE OF EACH SERVER-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(S), 0.03)

for i in range(len(S)):
  server1_latitude = S.iloc[i]['LATITUDE']
  server1_longitude = S.iloc[i]['LONGITUDE']
  temp = []
  for j in range(len(S)):
    server2_latitude = S.iloc[j]['LATITUDE']
    server2_longitude = S.iloc[j]['LONGITUDE']

    dist_geo = haversine(server1_longitude, server1_latitude, server2_longitude,server2_latitude)
    print(dist_geo)
    print(radius[i]+radius[j])
    if dist_geo <= radius[i]+radius[j]:
        temp.append(1)
        
    else:
        temp.append(0)
       
  U_si.append(temp)


0.0
0.06
0.025580942430779973
0.06
0.06399422245288403
0.06
0.05030372634034693
0.06
0.057270525900506325
0.06
0.08289693282665164
0.06
0.025580942430779973
0.06
0.0
0.06
0.08290865880948192
0.06
0.05263626769786789
0.06
0.04496028710605765
0.06
0.06905867767508603
0.06
0.06399422245288403
0.06
0.08290865880948192
0.06
0.0
0.06
0.04962559955657156
0.06
0.07908658792220558
0.06
0.09647506424093769
0.06
0.05030372634034693
0.06
0.05263626769786789
0.06
0.04962559955657156
0.06
0.0
0.06
0.029953928305510066
0.06
0.047697898055162884
0.06
0.057270525900506325
0.06
0.04496028710605765
0.06
0.07908658792220558
0.06
0.029953928305510066
0.06
0.0
0.06
0.025627615523929042
0.06
0.08289693282665164
0.06
0.06905867767508603
0.06
0.09647506424093769
0.06
0.047697898055162884
0.06
0.025627615523929042
0.06
0.0
0.06


In [ ]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in index:
    for i in range(len(S)):
        x[i, j] = model.NewBoolVar(f'c[{i}][{j}]')

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in range(len(S)):
    y[i] = model.NewBoolVar(f'c[{i}]')

print(model.ModelStats())

In [ ]:
#-----------------------------------CONSTRAINTS-----------------------------------

# Proximity constraint
for j in index:
    for i in range(len(S)):
        if U_si[i][j] == 0:
            model.Add(
             x[i, j]==0
            )

# Capacity constraint
for i in range(len(S)):
    for k in range(len(D)):
        model.Add(
            sum([
                x[i, j] * W[j][k] for j in index
            ]) <= C[i][k]*y[i]
        )

# Contraint family (each user can be allocated just once)
for j in index:
    model.Add(
        sum([
            x[i, j] for i in range(len(S))
        ]) <= 1
    )

print(model.ModelStats())

In [ ]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated users
objective_max = []
for j in index:
    for i in range(len(S)):
        objective_max.append(x[i, j])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_users = solver.ObjectiveValue()

# Hint (speed up solving)
for j in index:
    for i in range(len(S)):
        model.AddHint(x[i,j], solver.Value(x[i,j]))

for i in range(len(S)):
    model.AddHint(y[i], solver.Value(y[i]))
        
# Constraint previous objective
model.Add(
    sum([
        x[i, j] for i in range(len(S)) for j in index
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of servers used
objective_min = []
for i in range(len(S)):
    objective_min.append(y[i])
model.Minimize(sum(objective_min))

print(model.ModelStats())

In [ ]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    print(f'Objective value: {max_users} users have been allocated to {solver.ObjectiveValue()} servers\n')
    for j in index:
        for i in range(len(S)):
            if int(solver.Value(x[i,j])) == 1:
                print(f'User {j+1} has been allocated on server {i+1}')
else:
    print('The problem does not have an optimal solution.')

print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

In [ ]:
"""
#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are set manually

U_si = [] #set of individuals allocated to server si
D = np.zeros((4), dtype=int) # D = [CPU, RAM, storage, bandwidth]

#User u has memory requirements = w (example for 8 users)
w1 = [1, 2, 1, 2]
w2 = [1, 2,	1, 2]
w3 = [5, 7,	6, 6]
w4 = [2, 3,	3, 4]
w5 = [5, 7,	6, 6]
w6 = [2, 3,	3, 4]
w7 = [1, 2, 1, 2]
w8 = [2, 3,	3, 4]
w9 = [1, 2, 1, 2]
w10 = [1, 2, 1, 2]
w11 = [5, 7, 6, 6]
w12 = [2, 3, 3, 4]
w13 = [5, 7, 6, 6]
w14 = [2, 3, 3, 4]
W = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14]
#W = [w1, w2, w3, w4, w5, w6, w7, w8]

# Each server has memory capacity c (example for 3 servers)
c1 = [12, 15, 15, 16]
c2 = [10, 17, 15, 16]
c3 = [10, 15, 11, 15]
c4 = [10, 16, 15, 17]
c5 = [12, 11, 12, 13]
c6 = [9, 13, 12, 14]
C = [c1, c2, c3, c4, c5, c6]
#C = [c1, c2, c3]

NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8), LA.norm(w9), LA.norm(w10), LA.norm(w11), LA.norm(w12), LA.norm(w13), LA.norm(w14)]
#NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8)]

index = sorted(range(len(NORM)), key=lambda a: NORM[a]) #increasing order of the index of user based on NORM
 """

In [ ]:
'''
#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are randomly generated

U_si = [] #set of individuals allocated to server si
D = np.zeros((4), dtype=int) # D = [CPU, RAM, storage, bandwidth]

# Users u has memory requirements = w_j
W = []
NORM = []
for j in range(len(U)):
  w = np.random.randint(1,7,4)
  W.append(w)
  norm = LA.norm(w)
  NORM.append(norm)

index = sorted(range(len(NORM)), key=lambda a: NORM[a])

# Each server has memory capacity C
C = []
for j in range(len(S)):
  c = np.random.randint(9,17,4)
  C.append(c)
'''